## 数据挖掘简介

### 1. 数据挖掘的意义
    随着互联网、大数据时代的到来，我们的生活越来越离不开手机、电脑和网络，
    各项活动数据也会以日志的形式记录存放在各大服务方的数据库中，而这其中的有些数据很有潜在的挖掘价值。
    
    有人说数据就像新的能源，而数据挖掘就像提炼石油一样，尽可能的从中提炼出有用的信息，而这个有用信息更多的时候指的是预测。

### 2. 样本的构建与分析
   1. 特征工程
      1. 特征工程意义
      
           这些数据你可以理解成为一张张不同粒度的表，我们要做的就是从这些不同粒度来源的表中加工出 与我们建模样本粒度一致的字段（特征）。
           
           特征工程就是人为的设计X，通过人为的方式提高模型的表现，如果特征加工的好，简单的模型也能表现出优异的性能。
           
      2. 特征构建
         - 也就是怎么样将数据转化为特征，在这点上机器学习和统计学体现出来明显的差异，机器学习不管数据是怎么来的，更加侧重结果是什么，即以预测目标导向。
         
         - 主要在于构建思路，对业务有所了解会起到重要，对于程序实现，本次教学采用的是python的pandas库
         
      3. 特征提取
         - 主成分 embedding等降维方法
          
      4. 特征选择
         - 衡量特征重要性的指标；有些算法本身就会包含这一步，例如lasso、决策树。
         
         
   2. 标签处理
   
   3. 变量分析

### 3. 机器学习算法的选择与使用 
     LR、决策树、集成学习、深度学习等

---
## pandas数据处理函数的复习与补充

* .apply()
* pd.factorize()
* pd.get_dummies()
* .agg()
* pd.cut()
* pd.pivot_table()


In [1]:
import numpy as np
import pandas as pd

df=pd.DataFrame({'编码':list('12345678'),'姓名':['赵','钱','孙','李','赵','钱','孙','李'],
                 '性别':['男','女',np.nan,'女','男','女',np.nan,'女'],
                 '科目':['语文','语文','语文','语文','数学','数学','数学','数学'],
                 '分数':[5,3,7,8,7,4,9,8]})

In [2]:
df

,编码,姓名,性别,科目,分数
0,1,赵,男,语文,5
1,2,钱,女,语文,3
2,3,孙,NaN,语文,7
3,4,李,女,语文,8
4,5,赵,男,数学,7
5,6,钱,女,数学,4
6,7,孙,NaN,数学,9
7,8,李,女,数学,8


### 数据操作函数 .apply()

我们想增加一列作为总分

In [3]:
df['总分'] = df.apply(lambda x:df[df['姓名']==x.姓名]['分数'].sum(),axis=1)

df

,编码,姓名,性别,科目,分数,总分
0,1,赵,男,语文,5,12
1,2,钱,女,语文,3,7
2,3,孙,NaN,语文,7,16
3,4,李,女,语文,8,16
4,5,赵,男,数学,7,12
5,6,钱,女,数学,4,7
6,7,孙,NaN,数学,9,16
7,8,李,女,数学,8,16


### 数字编码函数 `pd.factorize()`

字符串特征转化为数字特征

In [4]:
data = df.copy()
data['性别'],uniques = pd.factorize(data['性别']) #空缺值默认转化为-1
uniques

Index(['男', '女'], dtype='object')

In [5]:
data 

,编码,姓名,性别,科目,分数,总分
0,1,赵,0,语文,5,12
1,2,钱,1,语文,3,7
2,3,孙,-1,语文,7,16
3,4,李,1,语文,8,16
4,5,赵,0,数学,7,12
5,6,钱,1,数学,4,7
6,7,孙,-1,数学,9,16
7,8,李,1,数学,8,16


### one-hot编码函数 `pd.get_dummies()`

我们想对性别进行one_hot编码

In [6]:
# 选择需要独热编码的字段
categorical_columns = ['性别']
# 是否将空值独热成一列
nan_as_category = True

pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)

,编码,姓名,科目,分数,总分,性别_女,性别_男,性别_nan
0,1,赵,语文,5,12,0,1,0
1,2,钱,语文,3,7,1,0,0
2,3,孙,语文,7,16,0,0,1
3,4,李,语文,8,16,1,0,0
4,5,赵,数学,7,12,0,1,0
5,6,钱,数学,4,7,1,0,0
6,7,孙,数学,9,16,0,0,1
7,8,李,数学,8,16,1,0,0


### 数据聚合方法 `.agg()` （常与`groupby`连用）

我们想对数据进行粒度的变化，转化以科目作为粒度

In [7]:
df.groupby('科目').agg({'分数':['size','max',('最小值','min')]})

分数        
   size max 最小值
科目             
数学    4   9   4
语文    4   8   3

### 分段映射函数 `pd.cut()`

我们想让分数根据不同分段映射为优良中差

In [8]:
# 设置分段点,默认左开右闭
bins = [0,5,7,9,10]

# 设置映射到的内容
labels = ['差','中','良','优']

pd.cut(df['分数'], bins, labels = labels)

0    差
1    差
2    中
3    良
4    中
5    差
6    良
7    良
Name: 分数, dtype: category
Categories (4, object): [差 < 中 < 良 < 优]

In [9]:
# 进而可以对其进行统计
pd.cut(df['分数'], bins, labels = labels).value_counts()

良    3
差    3
中    2
优    0
Name: 分数, dtype: int64

### 数据透视表函数 `pd.pivot_table()`

我们想对**不同学生**的**不同科目**的**分数**进行一些简单**统计**

In [10]:
pd.pivot_table(df,index=['姓名'],columns=['科目'],values=['分数'],aggfunc=['max','mean'])

max    mean   
    分数      分数   
科目  数学 语文   数学 语文
姓名               
孙    9  7    9  7
李    8  8    8  8
赵    7  5    7  5
钱    4  3    4  3